This noteboooks is for linear regression and neural networks model for stage two (predict top 10 growth rates suburb). However, these two models were not used in the final presentation due to data limitations and lack of accuracy.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
df = pd.read_csv("../data/curated/2016_data.csv")

In [ ]:
df_21 = pd.read_csv("../data/curated/2021_data.csv")

In [ ]:
bed_16 = pd.read_csv("../df_bed_2016.csv")
bed_21 = pd.read_csv("../df_bed_2021.csv")

bed_16 = bed_16.rename(columns={'SA2_2016': 'SA2_CODE16'})
bed_21 = bed_21.rename(columns={'SA2_2021': 'SA2_CODE16'})

In [ ]:
bed_21

In [ ]:
df

In [ ]:
df_21 = df_21.rename(columns={'SA2_CODE21': 'SA2_CODE16','Median_rent_weekly':'Median_rent_weekly_21'})

In [ ]:
# df_new = pd.merge(df, df_21[['SA2_CODE16','Median_rent_weekly_21']], how = 'left', on='SA2_CODE16')
df = pd.merge(df, bed_16,  on='SA2_CODE16')
df_new = pd.merge(df, df_21[['SA2_CODE16','Median_rent_weekly_21']],  on='SA2_CODE16')

In [ ]:
df_new.Median_rent_weekly_21.isnull().sum()

In [ ]:
df_new['label'] = (df_new['Median_rent_weekly_21']-df_new['Median_rent_weekly'])/(df_new['Median_rent_weekly']+1e-5)

In [ ]:
df_new

In [ ]:
df_train = df_new[(df_new.Median_rent_weekly != 0) & (df_new.avg_income.notna())]
df_train = df_train[df_train.Tot_P_P > 1000]

In [ ]:
df_train

In [ ]:
# df_train['Median_tot_fam_inc_weekly_avg_income_div'] = (df_train['Median_tot_fam_inc_weekly'] - df_train['avg_income']) / df_train['avg_income']
df_train['Median_age_persons_Tot_P_P_div'] = (df_train['Median_age_persons'] - df_train['Tot_P_P']) / df_train['Tot_P_P']


In [ ]:
df_train.isnull().any()

In [ ]:
df_train.corr()

In [ ]:
df_train.columns

In [ ]:
#Test Set
df_ceshi = df_21[['SA2_CODE16', 'avg_income', 'Tot_P_P','Median_age_persons']]
df_ceshi = pd.merge(df_ceshi, bed_21,  on='SA2_CODE16')

In [ ]:
# df_ceshi['Median_age_persons_Tot_P_P_div'] = (df_ceshi['Median_age_persons'] - df_ceshi['Tot_P_P']) / df_ceshi['Tot_P_P']

In [ ]:
df_ceshi = df_ceshi[(df_ceshi.avg_income.notna())]
df_ceshi = df_ceshi[df_ceshi.Tot_P_P > 1000]

In [ ]:
df_ceshi.isnull().any()

In [ ]:
df_ceshi

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
std_scaler = MinMaxScaler()
df_train_scaled = std_scaler.fit_transform(df_train[[f for f in df_train.columns if f not in 
                                            ['Median_rent_weekly','Median_rent_weekly_21',
                                             'Median_age_persons_Tot_P_P_div',
                                            'Median_tot_fam_inc_weekly', 'label']]])

In [ ]:
df_ceshi_scaled = std_scaler.fit_transform(df_ceshi[[f for f in df_ceshi.columns if f not in 
                                            ['Median_rent_weekly','Median_rent_weekly_21',
                                             'Median_age_persons_Tot_P_P_div',
                                            'Median_tot_fam_inc_weekly', 'label']]])

In [ ]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(df_train[['label']])

In [ ]:
df_train_scaled = pd.DataFrame(df_train_scaled)

In [ ]:
df_train_scaled

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train_scaled, y_train_scaled, 
                                                    test_size = 0.2, random_state = 23)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
from sklearn import linear_model

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train,y_train)

y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print('The MSE is:', mean_squared_error(y_test, y_pred))

print('The R2 is:', r2_score(y_test, y_pred))

print('The MAE is:', mean_absolute_error(y_test, y_pred))


In [ ]:
from sklearn.neural_network import MLPRegressor 

In [ ]:
clf = MLPRegressor(solver='lbfgs', alpha=1e-3, hidden_layer_sizes=(80,7), activation='relu', random_state=20)

clf.fit(X_train,y_train)

y_pred_2 = clf.predict(X_test)

In [ ]:
print('The R2 is:', r2_score(y_train, clf.predict(X_train)))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

print('The MSE is:', mean_squared_error(y_test, y_pred_2))

print('The R2 is:', r2_score(y_test, y_pred_2))

print('The MAE is:', mean_absolute_error(y_test, y_pred_2))

In [ ]:
clf.predict(X_test)

In [ ]:
cengindex = 0
for wi in clf.coefs_:
    cengindex += 1 
    print('%d:' % cengindex)
    print(wi.shape)
    print('\n',wi)


In [ ]:
# predict top 10 rate SA2:

In [ ]:
y_pred_mlp = clf.predict(df_ceshi_scaled)

In [ ]:
y_scaler.inverse_transform(y_pred_mlp.reshape((-1, 1)))

In [ ]:
y_pred_lr = lr.predict(df_ceshi_scaled)

In [ ]:
df_pred = pd.DataFrame()
df_pred['SA2_CODE21'] = df_ceshi['SA2_CODE16']
df_pred['ror_mlp_pred'] = y_scaler.inverse_transform(y_pred_mlp.reshape((-1, 1)))
df_pred['ror_lr_pred'] = y_scaler.inverse_transform(y_pred_lr.reshape((-1, 1)))

In [ ]:
df_pred.sort_values(["ror_mlp_pred"] , ascending=False) [:10]

In [ ]:
df_pred.sort_values(["ror_lr_pred"] , ascending=False)[:10]

In [ ]:
print("Top 10 growth rate SA2_CODE(MLP predict) are ",df_pred.sort_values(["ror_mlp_pred"] , ascending=False)[:10].SA2_CODE21.values)

In [ ]:
print("Top 10 growth rate SA2_CODE(LR predict) are ",df_pred.sort_values(["ror_lr_pred"] , ascending=False)[:10].SA2_CODE21.values)